<a href="https://colab.research.google.com/github/Dmitri9149/TensorFlow-PyTorch-basics/blob/master/TensorFlow_Blocks_and_Layesr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import tensorflow as tf

The code is based on the d2l.ai book http://d2l.ai/

In [60]:
### the code is based on d2l.ai book 

In [61]:
### sequence of models , Dense with relu is the first in the chain
net = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256,activation=tf.nn.relu),
  tf.keras.layers.Dense(10)
])

X=tf.random.uniform((2,20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.19088091, -0.24548021, -0.07908687,  0.30984086,  0.17110598,
         0.4218282 ,  0.12295652, -0.00994178,  0.21269724, -0.17230819],
       [-0.54047906, -0.04464073, -0.02348679,  0.13853593,  0.45006043,
         0.24140987,  0.397393  ,  0.02918926,  0.2661298 , -0.10237794]],
      dtype=float32)>

In [62]:
net.call(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.19088091, -0.24548021, -0.07908687,  0.30984086,  0.17110598,
         0.4218282 ,  0.12295652, -0.00994178,  0.21269724, -0.17230819],
       [-0.54047906, -0.04464073, -0.02348679,  0.13853593,  0.45006043,
         0.24140987,  0.397393  ,  0.02918926,  0.2661298 , -0.10237794]],
      dtype=float32)>

In [63]:
### Custom Block

In [64]:
class MLP(tf.keras.Model):
  def __init__(self):
    super().__init__()

    self.hidden=tf.keras.layers.Dense(units=256,activation=tf.nn.relu)
    self.out = tf.keras.layers.Dense(units=10)

  def call(self, X):
    return self.out(self.hidden((X)))

In [65]:
net=MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.1754027 ,  0.31547168,  0.02665147,  0.01312544, -0.02628017,
         0.2227064 , -0.2057536 ,  0.1793164 ,  0.00335142, -0.07642411],
       [-0.11393467,  0.22491458,  0.09532176,  0.04753569, -0.0741464 ,
        -0.03588762, -0.12225104,  0.0785155 ,  0.10170905, -0.3811512 ]],
      dtype=float32)>

In [66]:
class MySequential(tf.keras.Model):
  def __init__(self, *args):
    super().__init__()
    self.modules = []
    for block in args:
      self.modules.append(block)

  def call(self,X):
    for module in self.modules:
      X=module(X)
    return(X)


In [67]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.28162202, -0.04371443, -0.36139762,  0.47881842, -0.43754348,
        -0.29503652, -0.11193667,  0.30781952, -0.17923781,  0.25773805],
       [ 0.40401274, -0.17486447, -0.1706396 ,  0.32672796, -0.5723637 ,
        -0.2388013 , -0.09565606,  0.2968222 ,  0.01969995,  0.2901539 ]],
      dtype=float32)>

Below the Module is constructed where several models are applied in parallel on the same input data and the results are concatenated. It is intended to use the code in multihead attention in Transformers.

In [68]:
#initialize the list of n models which is constructed from the same model
#
def list_of_models(model, n):
  return [model]*n


In [69]:
list_models = [tf.keras.layers.Dense(10)]*3
arg_list = list_of_models(tf.keras.layers.Dense(10),3)

In [70]:
### use *args to supply a list of models of variable length 
class MyParallel(tf.keras.Model):
  def __init__(self, *args):
    super().__init__()
    self.modules = []
    for block in args:
      self.modules.append(block)

  def call(self,X):
    list_res= []
    for module in self.modules:
      list_res.append(module(X))
      print(len(list_res))
    print(list_res)
    concat_final= tf.concat(list_res, -1)
    return(concat_final)

In [71]:
net = MyParallel(tf.keras.layers.Dense(10), tf.keras.layers.Dense(10), tf.keras.layers.Dense(10))
X = tf.random.uniform((2,10))
net(X)

1
2
3
[<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.14490095,  0.0794435 ,  0.50017166, -0.6986517 ,  0.43846303,
         0.1231045 , -0.4789162 ,  0.00637756, -0.08736229, -1.2559376 ],
       [ 0.36836153, -0.55994344,  0.45377174, -0.6980738 ,  0.14614522,
        -0.43536675, -0.6988557 ,  0.6581564 ,  0.05491114, -1.1397681 ]],
      dtype=float32)>, <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.01551464,  0.17690885, -0.68878347, -0.44795692,  0.9871354 ,
         0.6054845 , -1.1978846 ,  0.45840088,  1.1669436 , -0.322392  ],
       [ 0.22445709, -0.1715191 , -0.35252792, -1.0359576 ,  1.0006998 ,
        -0.21304247, -1.0277196 ,  0.32946795,  1.2782837 , -0.5108025 ]],
      dtype=float32)>, <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.0067856 , -0.11662714,  0.54036695, -1.3106878 ,  0.3843606 ,
        -0.5279068 , -0.42929056,  1.0044799 , -0.79400826,  0.04439169],
       [ 0.15591703, -0.82232916, -0.09704366, -1.4714411 ,  0.

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[ 0.14490095,  0.0794435 ,  0.50017166, -0.6986517 ,  0.43846303,
         0.1231045 , -0.4789162 ,  0.00637756, -0.08736229, -1.2559376 ,
         0.01551464,  0.17690885, -0.68878347, -0.44795692,  0.9871354 ,
         0.6054845 , -1.1978846 ,  0.45840088,  1.1669436 , -0.322392  ,
         0.0067856 , -0.11662714,  0.54036695, -1.3106878 ,  0.3843606 ,
        -0.5279068 , -0.42929056,  1.0044799 , -0.79400826,  0.04439169],
       [ 0.36836153, -0.55994344,  0.45377174, -0.6980738 ,  0.14614522,
        -0.43536675, -0.6988557 ,  0.6581564 ,  0.05491114, -1.1397681 ,
         0.22445709, -0.1715191 , -0.35252792, -1.0359576 ,  1.0006998 ,
        -0.21304247, -1.0277196 ,  0.32946795,  1.2782837 , -0.5108025 ,
         0.15591703, -0.82232916, -0.09704366, -1.4714411 ,  0.81369925,
        -0.5280976 , -0.14407334,  0.9664606 , -1.4289265 , -0.06384918]],
      dtype=float32)>

In [72]:
### to use the list_models (which is a list) we have to unpack it
net = MyParallel(*list_models)
X = tf.random.uniform((2,10))
net(X)

1
2
3
[<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.2037656 ,  0.22916558,  0.70216995, -0.09731835, -0.32109368,
         0.7234111 , -0.088953  ,  0.41940314, -0.03820773,  0.11949056],
       [ 0.29042464,  0.70398897,  0.21444193, -0.34612215,  0.37787506,
         0.3378849 , -0.3878733 ,  0.12061216, -0.0404951 , -0.12163115]],
      dtype=float32)>, <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.2037656 ,  0.22916558,  0.70216995, -0.09731835, -0.32109368,
         0.7234111 , -0.088953  ,  0.41940314, -0.03820773,  0.11949056],
       [ 0.29042464,  0.70398897,  0.21444193, -0.34612215,  0.37787506,
         0.3378849 , -0.3878733 ,  0.12061216, -0.0404951 , -0.12163115]],
      dtype=float32)>, <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.2037656 ,  0.22916558,  0.70216995, -0.09731835, -0.32109368,
         0.7234111 , -0.088953  ,  0.41940314, -0.03820773,  0.11949056],
       [ 0.29042464,  0.70398897,  0.21444193, -0.34612215,  0.

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[-0.2037656 ,  0.22916558,  0.70216995, -0.09731835, -0.32109368,
         0.7234111 , -0.088953  ,  0.41940314, -0.03820773,  0.11949056,
        -0.2037656 ,  0.22916558,  0.70216995, -0.09731835, -0.32109368,
         0.7234111 , -0.088953  ,  0.41940314, -0.03820773,  0.11949056,
        -0.2037656 ,  0.22916558,  0.70216995, -0.09731835, -0.32109368,
         0.7234111 , -0.088953  ,  0.41940314, -0.03820773,  0.11949056],
       [ 0.29042464,  0.70398897,  0.21444193, -0.34612215,  0.37787506,
         0.3378849 , -0.3878733 ,  0.12061216, -0.0404951 , -0.12163115,
         0.29042464,  0.70398897,  0.21444193, -0.34612215,  0.37787506,
         0.3378849 , -0.3878733 ,  0.12061216, -0.0404951 , -0.12163115,
         0.29042464,  0.70398897,  0.21444193, -0.34612215,  0.37787506,
         0.3378849 , -0.3878733 ,  0.12061216, -0.0404951 , -0.12163115]],
      dtype=float32)>

In [73]:
net = MyParallel(*arg_list)
X = tf.random.uniform((2,10))
net(X)

1
2
3
[<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.5911249 , -0.52441454, -0.74450433,  0.7025785 ,  0.68058085,
        -0.20715527,  1.1025208 ,  0.9896529 ,  0.02379379, -0.5977524 ],
       [ 0.47362876, -0.99836683, -0.28144085,  1.1753883 ,  0.47592038,
         0.13608164,  0.89621174,  0.80639285,  0.01752793, -0.07582149]],
      dtype=float32)>, <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.5911249 , -0.52441454, -0.74450433,  0.7025785 ,  0.68058085,
        -0.20715527,  1.1025208 ,  0.9896529 ,  0.02379379, -0.5977524 ],
       [ 0.47362876, -0.99836683, -0.28144085,  1.1753883 ,  0.47592038,
         0.13608164,  0.89621174,  0.80639285,  0.01752793, -0.07582149]],
      dtype=float32)>, <tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.5911249 , -0.52441454, -0.74450433,  0.7025785 ,  0.68058085,
        -0.20715527,  1.1025208 ,  0.9896529 ,  0.02379379, -0.5977524 ],
       [ 0.47362876, -0.99836683, -0.28144085,  1.1753883 ,  0.

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[ 0.5911249 , -0.52441454, -0.74450433,  0.7025785 ,  0.68058085,
        -0.20715527,  1.1025208 ,  0.9896529 ,  0.02379379, -0.5977524 ,
         0.5911249 , -0.52441454, -0.74450433,  0.7025785 ,  0.68058085,
        -0.20715527,  1.1025208 ,  0.9896529 ,  0.02379379, -0.5977524 ,
         0.5911249 , -0.52441454, -0.74450433,  0.7025785 ,  0.68058085,
        -0.20715527,  1.1025208 ,  0.9896529 ,  0.02379379, -0.5977524 ],
       [ 0.47362876, -0.99836683, -0.28144085,  1.1753883 ,  0.47592038,
         0.13608164,  0.89621174,  0.80639285,  0.01752793, -0.07582149,
         0.47362876, -0.99836683, -0.28144085,  1.1753883 ,  0.47592038,
         0.13608164,  0.89621174,  0.80639285,  0.01752793, -0.07582149,
         0.47362876, -0.99836683, -0.28144085,  1.1753883 ,  0.47592038,
         0.13608164,  0.89621174,  0.80639285,  0.01752793, -0.07582149]],
      dtype=float32)>

In [74]:
### Nested Models 
class NestedBlocks(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.net=tf.keras.Sequential()
    self.net.add(tf.keras.layers.Dense(20,activation = tf.nn.relu))
    self.net.add(tf.keras.layers.Dense(20,activation = tf.nn.relu))
    self.dense = tf.keras.layers.Dense(20,activation = tf.nn.relu)

  def call(self, inputs):
    return self.dense(self.net(inputs))



In [75]:
net = NestedBlocks()
X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[0.03311471, 0.        , 0.15942249, 0.        , 0.08883218,
        0.22490145, 0.        , 0.21377383, 0.03714283, 0.        ,
        0.        , 0.        , 0.1599341 , 0.10077477, 0.        ,
        0.17028964, 0.        , 0.30675215, 0.10723516, 0.        ],
       [0.        , 0.15211217, 0.09624891, 0.        , 0.        ,
        0.38231185, 0.        , 0.23157892, 0.01519379, 0.        ,
        0.        , 0.00758522, 0.14658591, 0.08670777, 0.0599402 ,
        0.28656822, 0.        , 0.31177   , 0.06682221, 0.0498123 ]],
      dtype=float32)>